In [ ]:
using DiffEqFlux, DifferentialEquations, Plots, DelimitedFiles

In [3]:
N = 10^7
dat = readdlm("covid.txt",',');
time_i = dat[:,1]; # times
Infected = 50*dat[:,2]/N; # Normalize data
Infected[2:end] = Infected[2:end];
A = Infected;

tshort_inds = (Int.(time_i)*10).+1
tshort = Int.(time_i).+1;

In [24]:
#tsteps = range(tspan[1], tspan[2], length = datasize)
tspan = (0.0, maximum(time_i))
tsteps = time_i
#=
function trueODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u.^3)'true_A)'
end

prob_trueode = ODEProblem(trueODEfunc, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(), saveat = tsteps))
ode_data = ode_data[1,:]
=#
ode_data = Infected
u0 = Float32[0.0; 0.0; 0.0; 0.0]


dudt2 = FastChain(FastDense(4, 100, relu),
                FastDense(100, 50, relu),
                FastDense(50, 4))
prob_neuralode = NeuralODE(dudt2, tspan, Vern7(), saveat = tsteps, abstol=1e-6, reltol=1e-6)

function predict_neuralode(p)
  Array(prob_neuralode(u0, p))
end

function loss_neuralode(p)
    global tsteps
    pred = predict_neuralode(p)
    k = size(pred,2)
    t_loc = Int.(tsteps[tsteps .<= k] .+1)
    #lloss = sum(abs2, (ode_data[1:size(pred,2)] .- pred[1,:]))
    loss = sum(abs2, (ode_data[1:length(t_loc)] .- pred[1,t_loc]))
    return loss, pred
end

iter = 0
name_vorlage = "video/NODE_realdata_im"
callback = function (p, l, pred; doplot = true)
  global iter
  global tsteps
  iter += 1
  im_name = name_vorlage*string(iter)*".png"
  display(iter)
  display(l)
  if doplot
    # plot current prediction against data
    k = size(pred,2)
    t_local = tsteps[tsteps .<= k]
    plt = scatter(t_local, ode_data[1:length(t_local)], 
            label = "Data", 
            ylabel="Infected/N",
            xlabel="t",
            legend=:outertopright, 
            size=(1500,1000),
            dpi=500,
            xtickfont=font(20),
            ytickfont=font(20),
            yguidefontsize=18,
            xguidefontsize=18,
            legendfontsize=20,
            titlefontsize=20,
            markersize=10,
            left_margin=10Plots.mm,
            bottom_margin=10Plots.mm)
    scatter!(plt, 0:k-1, pred[1,:], label = "Neural ODE", markersize=10)
    #display(plot(plt))
    savefig(im_name)
  end
  return false
end

#16 (generic function with 1 method)

In [25]:
t4 = 60.0
prob_neuralode = NeuralODE(dudt2, (0.0,t4), Tsit5(), saveat = 0:t4)
result_neuralode5 = DiffEqFlux.sciml_train(loss_neuralode,
                                           #result_neuralode5.u,
                                           prob_neuralode.p,
                                           ADAM(), maxiters = 1000,
                                           cb = callback)

1

0.0035319153750000003

2

54347.819708386676

3

...

1000

4.2068781874117965e-5

1001

4.091112035404471e-5

u: 5754-element Vector{Float32}:
  0.13736413
 -0.13768275
 -0.17891036
 -0.026071616
 -0.045979116
 -0.013184977
 -0.13118507
 -0.047984995
  0.19676572
 -0.19964251
  0.11598197
  0.21375751
 -0.07777536
  ⋮
 -0.26298568
 -0.15585859
  0.048815735
 -0.15172441
 -0.1896129
  0.17747149
 -0.32454064
 -0.23978494
 -0.006521153
 -0.0067135496
  0.0045597265
 -0.0045235604

In [26]:
t5 = 91.0
prob_neuralode = NeuralODE(dudt2, (0.0,t5), Tsit5(), saveat = 0:t5)
result_neuralode6 = DiffEqFlux.sciml_train(loss_neuralode,
                                           result_neuralode5.u,
                                           ADAM(), maxiters = 1000,
                                           cb = callback)

1002

0.8228363546356289

1003

0.11451570187602464

1004

...

2001

0.0010479987033201446

2002

0.0010410320862945346

u: 5754-element Vector{Float32}:
  0.14428839
 -0.13768275
 -0.17891036
 -0.032025658
 -0.038367733
 -0.021223662
 -0.109136716
 -0.054638688
  0.20343892
 -0.19250897
  0.11962086
  0.2068442
 -0.07777536
  ⋮
 -0.26298568
 -0.15585859
  0.048815735
 -0.15172441
 -0.1896129
  0.17747149
 -0.32454064
 -0.23978494
 -0.004593029
 -0.013021954
  0.0036018444
  0.0003960597

In [27]:
t6 = 103.0
prob_neuralode = NeuralODE(dudt2, (0.0,t6), Tsit5(), saveat = 0:t6)
result_neuralode7 = DiffEqFlux.sciml_train(loss_neuralode,
                                           result_neuralode6.u,
                                           ADAM(), maxiters = 1000,
                                           cb = callback)

2003

0.10396856748304184

2004

0.5044758175072847

2005

...

3002

0.0017670637914394367

3003

0.0005122553155372708

u: 5754-element Vector{Float32}:
  0.15131733
 -0.13768275
 -0.17891036
 -0.021488225
 -0.038367733
 -0.015876625
 -0.113107175
 -0.054638688
  0.2110613
 -0.18616426
  0.11280871
  0.2068442
 -0.07777536
  ⋮
 -0.26298568
 -0.15585859
  0.048815735
 -0.15172441
 -0.1896129
  0.17747149
 -0.32454064
 -0.23978494
 -0.0066124713
 -0.0075650364
  0.0062742536
 -0.0016755912

In [28]:
t7 = 134.0
prob_neuralode = NeuralODE(dudt2, (0.0,t7), Tsit5(), saveat = 0:t7)
result_neuralode8 = DiffEqFlux.sciml_train(loss_neuralode,
                                           result_neuralode7.u,
                                           ADAM(), maxiters = 1000,
                                           cb = callback)

3004

0.03034768053665958

3005

0.2730347053437383

3006

...

4003

0.00027333465125179964

4004

0.00018542864349061793

u: 5754-element Vector{Float32}:
  0.14156465
 -0.13768275
 -0.17891036
 -0.031390253
 -0.038367733
 -0.033787638
 -0.09066402
 -0.06328777
  0.26020777
 -0.17337783
  0.1060089
  0.21101613
 -0.07777536
  ⋮
 -0.26298568
 -0.15585859
  0.048815735
 -0.15172441
 -0.1896129
  0.17747149
 -0.32454064
 -0.23978494
 -0.005830961
 -0.017322456
  2.3111372f-5
 -0.0037944028

In [29]:
t8 = 149.0
prob_neuralode = NeuralODE(dudt2, (0.0,t8), Tsit5(), saveat = 0:t8)
result_neuralode9 = DiffEqFlux.sciml_train(loss_neuralode,
                                           result_neuralode8.u,
                                           ADAM(), maxiters = 1000,
                                           cb = callback)

4005

0.00028079410473704034

4006

0.8986110042049194

4007

...

5004

0.00043432754595173345

5005

0.00028079410473704034

u: 5754-element Vector{Float32}:
  0.14156465
 -0.13768275
 -0.17891036
 -0.031390253
 -0.038367733
 -0.033787638
 -0.09066402
 -0.06328777
  0.26020777
 -0.17337783
  0.1060089
  0.21101613
 -0.07777536
  ⋮
 -0.26298568
 -0.15585859
  0.048815735
 -0.15172441
 -0.1896129
  0.17747149
 -0.32454064
 -0.23978494
 -0.005830961
 -0.017322456
  2.3111372f-5
 -0.0037944028

In [30]:
t9 = 170.0
prob_neuralode = NeuralODE(dudt2, (0.0,t9), Tsit5(), saveat = 0:t9)
result_neuralode10 = DiffEqFlux.sciml_train(loss_neuralode,
                                           result_neuralode9.u,
                                           ADAM(), maxiters = 1000,
                                           cb = callback)

5006

0.010593144427413704

5007

0.640866055224549

5008

...

6005

0.00022059538945746484

6006

0.00019691781821843353

u: 5754-element Vector{Float32}:
  0.15276758
 -0.14531726
 -0.17891036
 -0.034662344
 -0.043926597
 -0.05109332
 -0.0907166
 -0.06749599
  0.25613743
 -0.17084204
  0.10890484
  0.20742224
 -0.03776668
  ⋮
 -0.26298568
 -0.15585859
  0.048815735
 -0.15172441
 -0.1896129
  0.17747149
 -0.32454064
 -0.23978494
 -0.00449588
 -0.016802415
 -0.0015634119
  0.0014838484